<h2> Tensorflow: Convolutional NN </h2>

In [0]:
影片未完成

<br> Components Of the Model
* Model Function(cnn_model_fn)
    + 將 特徵、分類標籤、模式和模組當作參數
    + 定義神經層
    + 定義輸出預測的字典
    + 創造 EstimatorSpec 給適當的模式
        - train, predict, eval
        - 產生一個 one hot 類別給 train and eval(評估)
        - eval 需要使用到量測的dict
* Main Function(main)
    + 將 模式和模組當作參數
    + 由我們的自訂函式取得資料
        - load MNIST from TF in this case
    + 由 cnn_model 和 模組參數 產生預估器
    + 產生不同模式的輸入函示(例如:train)
        - 從 TensorFlow 的 API numpy_input_fn 給 numpy 的資料
    + 用適當的模式跑分類(預估)器

In [1]:
import numpy
import tensorflow as tf

<h3> 設立訓練的 log </h3>
<p><a href="https://blog.csdn.net/vagrantabc2017/article/details/77507168" target="_blank" rel="noopener">tensorflow訓練過程與日誌監控</a></p>

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

<h3>模型function Model Function(cnn_model_fn)

In [0]:
###### CNN 模組 ###
def cnn_model_fn(features, labels, modem, params):
    ## 輸入層
    #  Reshape X to 4-D tensor : [batch_size, width, height, channels]
    #  手寫資料都是 28 X 28 pixels 單色(one color channel)
    input_layer = tf.reshape(feature["x"], [-1, 28, 28, 1]) # [-1, 28, 28, 1] 第一個-1就會動態 reshape 成 batch_size
    
    ## 第一層卷積層 計算32個特徵使用 5 X 5 的濾網，使用Relu 激活函式
    # 輸入 Tensor shape: [batch_size, 28, 28, 1]
    # 輸出 Tensor shape: [batch_size, 28, 28, 32(filters的size)]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernal_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    ## 第一個 max pooling layer with a 2 X 2 filter and stride of 2
    # 輸入 Tensor shape: [batch_size, 28, 28, 32]
    # 輸出 Tensor shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    ## 第二層卷積層 計算64個特徵使用 5 X 5 的濾網
    # 輸入 Tensor shape: [batch_size, 14, 14, 32]
    # 輸出 Tensor shape: [batch_size, 14, 14, 64]
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernal_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    ## 第二個 max pooling layer 
    # 輸入 Tensor shape: [batch_size, 14, 14, 64]
    # 輸出 Tensor shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    ## 平坦層 
    # 把最後 max pooling layer 壓平，準備輸入神經層網路
    pool2_flat =  tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    # 輸入 Tensor shape: [batch_size, 7 * 7 * 64]
    # 輸出 Tensor shape: [batch_size, 1024]    
    dense = tf.layers.dense(inpoputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    ## Add dropout operation;60% 比例的神經元會留下來
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training= mode == tf.estimator.ModeKeys.TRAIN)
    
    ## Logits layer
    # 輸入 Tensor shape: [batch_size, 1024]
    # 輸出 Tensor shape: [batch_size, 10]
    logits = tf.layers.dense(inpoputs=dropout, units=10)
    
    ## 這個dict是預測的結果，會回傳出去
    predictions = {
        # 預測的種類
        "classes": tf.argmax(input=logits, axis=1).
        # 用softmax 取得分類機率
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    ## 計算 Loss (for both Train and EVAL modes)
    onehot_lables = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_lables, logits=logits)
    
    ## 建構訓練優化器(for TRAIN 模式)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
   
    ## 評估指標 (for EVAL 模式)
    eval_metric_ops = {"accuracy":tf.metrics.accuracy(labels=labels, pedictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

<h3> Main Function

In [0]:
def main(mode="train", model_params={"learning_rate":0.001}):
    ## 載入訓練資料以及評估資料
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    
    # Train
    train_data = mnist.train.images  ## Return np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    
    # Test
    eval_data  = mnist.test.images
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # 創造評估器
    mnist_classifier = tf.estimator.Estimator()